In [59]:
import web3
from web3 import Web3

w3test=Web3(Web3.HTTPProvider('https://ropsten.infura.io/v3/c8930e58ceb84ae88155d2e842485535'))
w3=Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/2d964d9d1a614bce9979872979fcb0ae'))

import json
import os
fileDir = os.path.dirname(os.path.realpath('__file__'))
from hexbytes import HexBytes

In [60]:
start_block =  12099123
contract_address = '0x2Fc0E2Cfe5D6Ea300D555E5907319a5F7E09884f'
test_wallet = '0xFAD55dbF3B40289338454fDcd240995997fda751'
construction_address = '0xa72d7ad36f370a35ee5012c88d06dfa9f9937c9ab7012c44f20ac44994e884e4'

multisig = '0xF5581dFeFD8Fb0e4aeC526bE659CFaB1f8c781dA'

In [61]:
block = w3.eth.get_block('latest')

In [62]:
w3test.eth.get_balance(test_wallet)

4000000000000000000

In [63]:
#1) get abi 2) create contract object
path = os.path.join(fileDir, 'abi/HoprFarm.json')
with open(path) as json_file:
    myabi = json.load(json_file)
    #print(myabi)
    
contract = w3.eth.contract(address = '0x2Fc0E2Cfe5D6Ea300D555E5907319a5F7E09884f',
                              abi = myabi)

In [6]:
# Get Transactions option 1 (very slow): iterate over all blocks
for i in range(1):
    block = w3.eth.get_block(start_block+i)
    for tx_hash in block.transactions:
        receipt = w3.eth.get_transaction_receipt(tx_hash)
        if receipt.to == contract_address:
            objLog = contract.events.TokenAdded().processReceipt(receipt)
            print(objLog)

KeyboardInterrupt: 

In [64]:
# Get Transactions option 2 (good): using Filters
filt = contract.events.TokenAdded.createFilter(fromBlock=start_block, address=contract_address)
log_TokenAdded = w3.eth.get_filter_logs(filt.filter_id)
filt = contract.events.TokenRemoved.createFilter(fromBlock=start_block, address=contract_address)
log_TokenRemoved = w3.eth.get_filter_logs(filt.filter_id)
filt = contract.events.IncentiveClaimed.createFilter(fromBlock=start_block, address=contract_address)
log_IncentiveClaimed = w3.eth.get_filter_logs(filt.filter_id)

In [65]:
filt = contract.events.TokenAdded.createFilter(fromBlock=start_block, address=contract_address)
filt.get_all_events()
#log_TokenAdded = w3.eth.get_filter_logs(filt.filter_id)

AttributeError: 'LogFilter' object has no attribute 'get_all_events'

In [112]:
a = json.loads(jsonString)
b = a[1]['topics'][1]
print(b)
print(int(b, 16))
print(hex(int(b, 16)))



0x00000000000000000000000035c49bfe4fb19c46d14d229be0826865d033b686
306961045854547335554311754591700453332578711174
0x35c49bfe4fb19c46d14d229be0826865d033b686


In [1]:
# We need a custom JSONEncoder to encode HexBytes objects to json
class HexEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, HexBytes):
            return obj.hex()
        return json.JSONEncoder.default(self, obj)
    
json.dumps(dict(log_TokenAdded[0]), cls=HexEncoder)

# No, we don't! w3 has a JSON Encoder that does everything we need
w3.toJSON(log_TokenAdded)

NameError: name 'json' is not defined

In [10]:
# Option 1 (bad): using custom JSON Encoder
with open("data/TokenAdded.json", 'w') as f:
    json.dump([dict(a) for a in log_TokenAdded], f, cls=HexEncoder)
with open("data/TokenRemoved.json", 'w') as f:
    json.dump([dict(a) for a in log_TokenRemoved], f, cls=HexEncoder)
with open("data/IncentiveClaimed.json", 'w') as f:
    json.dump([dict(a) for a in log_IncentiveClaimed], f, cls=HexEncoder)

NameError: name 'HexEncoder' is not defined

In [66]:
# Option 2 (prefered): Using JSON Encoder from w3
with open("data/TokenAdded.json", 'w') as f:
    jsonString = w3.toJSON(log_TokenAdded)
    f.write(jsonString)
with open("data/TokenRemoved.json", 'w') as f:
    jsonString = w3.toJSON(log_TokenRemoved)
    f.write(jsonString)
with open("data/IncentiveClaimed.json", 'w') as f:
    jsonString = w3.toJSON(log_IncentiveClaimed)
    f.write(jsonString)

In [21]:
log_

TypeError: list indices must be integers or slices, not str

In [ ]:
# Next Goal: Finding out constructor parameters from creator transaction

In [40]:
# w3.eth.get_transaction_receipt(construction_address)
construction_transaction = w3.eth.get_transaction(construction_address)
# contract.decode_function_input('')
# contract.decode_function_input(construction_transaction.input)
construction_transaction.input
# contract.decode_function_input(w3.eth.get_transaction('0xefb61ce0fdea820289207edea0fb69d8e9f25d6b6963d7f5f379242682efe299').input)
contract.constructor

<bound method Contract.constructor of <class 'web3._utils.datatypes.Contract'>>

In [14]:
contract.all_functions()



[<Function TOTAL_CLAIM_PERIOD()>,
 <Function TOTAL_INCENTIVE()>,
 <Function WEEKLY_BLOCK_NUMBER()>,
 <Function WEEKLY_INCENTIVE()>,
 <Function claimAndClose()>,
 <Function claimFor(address)>,
 <Function claimedIncentive()>,
 <Function closeFarm(uint256)>,
 <Function currentFarmIncentive(uint256)>,
 <Function currentFarmPeriod()>,
 <Function distributionBlocks(uint256)>,
 <Function eligibleLiquidityPerPeriod(uint256)>,
 <Function hopr()>,
 <Function incentiveToBeClaimed(address)>,
 <Function liquidityProviders(address)>,
 <Function multisig()>,
 <Function openFarm(uint256)>,
 <Function openFarmWithPermit(uint256,address,uint256,uint8,bytes32,bytes32)>,
 <Function pool()>,
 <Function recoverToken(address)>,
 <Function tokensReceived(address,address,address,uint256,bytes,bytes)>,
 <Function totalPoolBalance()>]

In [19]:
[contract.functions.distributionBlocks(i).call() for i in range(14)]

[12141500,
 12186300,
 12231100,
 12275900,
 12320700,
 12365500,
 12410300,
 12455100,
 12499900,
 12544700,
 12589500,
 12634300,
 12679100,
 12723900]